In [1]:
import pandas as pd
import numpy as np

import os
import pathlib
import importlib

from sklearn.model_selection import train_test_split

In [2]:
def create_dir_if_not_exists(path):
    if not pathlib.Path(path).exists():
        os.mkdir(path)

def create_train_val_test_dir(path):
    for dir in ['train', 'val', 'test']:
        create_dir_if_not_exists('{}/{}'.format(path, dir))

In [3]:
# Directory containing the OAS files
OAS_FILE_DIR = r'../../datasets/OAS_sample'

DATASET_DIRECTORY = r'../datasets/'
create_dir_if_not_exists(DATASET_DIRECTORY)

# Directory to store the "raw" sequences
SEQUENCES_DIRECTORY = '{}/sequences'.format(DATASET_DIRECTORY)
create_dir_if_not_exists(SEQUENCES_DIRECTORY)

PAIRED_SEQUENCES_FILE = '{}/sequences.csv'.format(SEQUENCES_DIRECTORY)
ONLY_REPRESENTATIVE = '{}/representative.csv'.format(SEQUENCES_DIRECTORY)

SEQUENCES_TRAIN = '{}/train.csv'.format(SEQUENCES_DIRECTORY)
SEQUENCES_VAL = '{}/val.csv'.format(SEQUENCES_DIRECTORY)
SEQUENCES_TEST = '{}/test.csv'.format(SEQUENCES_DIRECTORY)

SEQUENCES_FILES = {
    'train': SEQUENCES_TRAIN,
    'val': SEQUENCES_VAL,
    'test': SEQUENCES_TEST
}

# Directory to store the fasta files
FASTA_DIRECTORY = '{}/fasta'.format(DATASET_DIRECTORY)
create_dir_if_not_exists(FASTA_DIRECTORY)

# Directory to store the clustering files
CLUSTERING_DIRECTORY = '{}/clustering'.format(DATASET_DIRECTORY)
create_dir_if_not_exists(CLUSTERING_DIRECTORY)

CLASSIFICATOR_DIR = '{}/classificator'.format(DATASET_DIRECTORY)
create_dir_if_not_exists(CLASSIFICATOR_DIR)

RANDOM_DATASET_DIR = '{}/random'.format(CLASSIFICATOR_DIR)
create_dir_if_not_exists(RANDOM_DATASET_DIR)
create_train_val_test_dir(RANDOM_DATASET_DIR)

GERMLINE_ALL_DIR = '{}/germline_all'.format(CLASSIFICATOR_DIR)
create_dir_if_not_exists(GERMLINE_ALL_DIR)
create_train_val_test_dir(GERMLINE_ALL_DIR)

GERMLINE_V_DIR = '{}/germline_v'.format(CLASSIFICATOR_DIR)
create_dir_if_not_exists(GERMLINE_V_DIR)
create_train_val_test_dir(GERMLINE_V_DIR)

# Directory to store the data for additional tests
TEST_DIRECTORY = '{}/test'.format(DATASET_DIRECTORY)
create_dir_if_not_exists(TEST_DIRECTORY)

### Read raw data

### read_raw
Process a set of OAS data chunk creating a single csv file.

Arguments
- location: directory containg the OAS source files
- output_location: output file path
- subsample: sample a subset of the OAS source files found in location, default None
- only_human: focus only on human OAS source files
- 
TODO?



In [4]:
import read_raw

In [5]:
WHICH_GERMLINE = 'all'
STORE_SPECIE = False
SUBSAMPLE = None
ONLY_HUMAN = True

In [6]:
COMPUTE_NEW = True

In [7]:
if COMPUTE_NEW:
    importlib.reload(read_raw)
    read_raw.read_raw(OAS_FILE_DIR, PAIRED_SEQUENCES_FILE, 
                      subsample=SUBSAMPLE, only_human=ONLY_HUMAN,
                      which_germline=WHICH_GERMLINE, store_specie=STORE_SPECIE)

Reading data...
Found 16 files.
human:                 14
rat_SD:                 2


100%|██████████| 14/14 [00:24<00:00,  1.75s/it]


Filtering the data
Initial number of rows: 87116
Removed 50301 rows (-57.740%), new number of rows: 36815.
Assining ids...
Number of unique heavy: 36555
Number of unique light: 28518
Number of unique pairs:  36813
Cleaning the germlines...
Saved: ../datasets//sequences/sequences.csv


### Clusterize the sequences

In [8]:
import generate_fasta

In [9]:
WHICH = 'both'

if WHICH != 'both':
    FASTA_SEQUENCES = 'sequences_{}.fasta'.format(WHICH)
else:
    FASTA_SEQUENCES = 'sequences.fasta'

In [10]:
COMPUTE_NEW = True

In [11]:
if COMPUTE_NEW:
    importlib.reload(generate_fasta)
    generate_fasta.generate_fasta(PAIRED_SEQUENCES_FILE, 
                                  '{}/{}'.format(FASTA_DIRECTORY, FASTA_SEQUENCES))

100%|██████████| 36814/36814 [00:03<00:00, 9547.35it/s] 


Saved: ../datasets//fasta/sequences.fasta


In [12]:
MIN_SEQ_ID = 0.8

commands = 'source cluster.sh {} {} {} {}\n'.format(
    DATASET_DIRECTORY, 
    'fasta/{}'.format(FASTA_SEQUENCES), 
    'clustering/sequences', 
    MIN_SEQ_ID
)

commands += 'rm -rf {}/fasta_files'.format(DATASET_DIRECTORY)

with open('clustering_commands.sh', 'w') as f:
    f.write(commands)

In [13]:
os.system('chmod +x clustering_commands.sh')
os.system('bash ./clustering_commands.sh')

createdb ../datasets//fasta/sequences.fasta ../datasets//DB/DB 

MMseqs Version:       	62975ca936b912083c2218e4e30ad962901cbb3b
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[===
Time for merging to DB_h: 0h 0m 0s 16ms
Time for merging to DB: 0h 0m 0s 26ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 327ms
Create directory ../datasets//cluster/tmp
linclust ../datasets//DB/DB ../datasets//cluster/cluster ../datasets//cluster/tmp --min-seq-id 0.8 

MMseqs Version:                     	62975ca936b912083c2218e4e30ad962901cbb3b
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	4
Compressed                          	0
Verbosity                           	3
Weight file name                    	
Cluster Weight threshold       

0

### Get only representative

In [14]:
import get_representative

In [15]:
COMPUTE_NEW = True

In [16]:
if COMPUTE_NEW:
    importlib.reload(get_representative)
    get_representative.get_representative('{}/sequences.tsv'.format(CLUSTERING_DIRECTORY), 
                                          PAIRED_SEQUENCES_FILE, ONLY_REPRESENTATIVE)

Extracting only the representative sequences...
Saved: ../datasets//sequences/representative.csv


### Split in train, val and test

In [17]:
import split

In [18]:
COMPUTE_NEW = True

In [19]:
if COMPUTE_NEW:
    importlib.reload(split)
    split.split(ONLY_REPRESENTATIVE, SEQUENCES_TRAIN, SEQUENCES_VAL, SEQUENCES_TEST)

Start splitting
Save training split: ../datasets//sequences/train.csv
Save validation split: ../datasets//sequences/val.csv
Save test split: ../datasets//sequences/test.csv


### Get germline files

In [20]:
import get_germlines

In [21]:
COMPUTE_NEW = True

In [ ]:
# Aggiunge il pairing ID
if COMPUTE_NEW:
    importlib.reload(get_germlines)
    for key, value in SEQUENCES_FILES.items():
        get_germlines.get_germlines(
            value, 
            '{}/{}/{}_seq.csv'.format(GERMLINE_V_DIR, key, key),
            '{}/{}/{}_germ.csv'.format(GERMLINE_V_DIR, key, key),
            which='v'
        )

Get germlines id...
Number of unique heavy combinations: 7
Number of unique light combinations: 17
Number of possibile heavy and light combinations: 119
Saved: ../datasets//classificator/germline_v/train/train_seq.csv, ../datasets//classificator/germline_v/train/train_germ.csv
Get germlines id...
Number of unique heavy combinations: 7
Number of unique light combinations: 16
Number of possibile heavy and light combinations: 112
Saved: ../datasets//classificator/germline_v/val/val_seq.csv, ../datasets//classificator/germline_v/val/val_germ.csv
Get germlines id...
Number of unique heavy combinations: 7
Number of unique light combinations: 16
Number of possibile heavy and light combinations: 112
Saved: ../datasets//classificator/germline_v/test/test_seq.csv, ../datasets//classificator/germline_v/test/test_germ.csv


In [46]:
if COMPUTE_NEW:
    importlib.reload(get_germlines)
    for key, value in SEQUENCES_FILES.items():
        get_germlines.get_germlines(
            value, 
            '{}/{}/{}_seq.csv'.format(GERMLINE_ALL_DIR, key, key),
            '{}/{}/{}_germ.csv'.format(GERMLINE_ALL_DIR, key, key),
            which='all'
        )

Get germlines id...
Number of unique heavy combinations: 225
Number of unique light combinations: 59
Number of possibile heavy and light combinations: 13275
Saved: ../datasets//classificator/germline_all/train/train_seq.csv, ../datasets//classificator/germline_all/train/train_germ.csv
Get germlines id...
Number of unique heavy combinations: 207
Number of unique light combinations: 59
Number of possibile heavy and light combinations: 12213
Saved: ../datasets//classificator/germline_all/val/val_seq.csv, ../datasets//classificator/germline_all/val/val_germ.csv
Get germlines id...
Number of unique heavy combinations: 225
Number of unique light combinations: 62
Number of possibile heavy and light combinations: 13950
Saved: ../datasets//classificator/germline_all/test/test_seq.csv, ../datasets//classificator/germline_all/test/test_germ.csv


### Germline pairing

In [40]:
import germline_pairing

In [41]:
ALPHA = 1000
NUMBER = None

In [42]:
COMPUTE_NEW = True

In [48]:
if COMPUTE_NEW:
    importlib.reload(germline_pairing)
    for split in ['train', 'val', 'test']:
        germline_pairing.germline_pairing(
            '{}/{}/{}_seq.csv'.format(GERMLINE_ALL_DIR, split, split),
            '{}/{}/{}_germline_pairing_alpha-{}.csv'.format(GERMLINE_ALL_DIR, split, split, ALPHA),
            '{}/{}/{}_germ'.format(GERMLINE_ALL_DIR, split, split),
            NUMBER, ALPHA
        )

10361 out of 13275 of only zeros


100%|██████████| 10361/10361 [00:00<00:00, 11796.39it/s]


Retrieve sequences


100%|██████████| 5965/5965 [00:02<00:00, 2053.84it/s]


Saved: ../datasets//classificator/germline_all/train/train_germline_pairing_alpha-1000.csv
10033 out of 12213 of only zeros


...

Saved: ../datasets//classificator/germline_all/val/val_germline_pairing_alpha-1000.csv
11311 out of 13950 of only zeros


100%|██████████| 11311/11311 [00:01<00:00, 7298.62it/s]


Retrieve sequences


100%|██████████| 4889/4889 [00:04<00:00, 1048.31it/s]


Saved: ../datasets//classificator/germline_all/test/test_germline_pairing_alpha-1000.csv


In [49]:
if COMPUTE_NEW:
    importlib.reload(germline_pairing)
    for split in ['train', 'val', 'test']:
        germline_pairing.germline_pairing(
            '{}/{}/{}_seq.csv'.format(GERMLINE_V_DIR, split, split),
            '{}/{}/{}_germline_pairing_alpha-{}.csv'.format(GERMLINE_V_DIR, split, split, ALPHA),
            '{}/{}/{}_germ'.format(GERMLINE_V_DIR, split, split),
            NUMBER, ALPHA
        )

22 out of 119 of only zeros


100%|██████████| 22/22 [00:00<00:00, 1788.34it/s]


Retrieve sequences


100%|██████████| 22/22 [00:00<00:00, 529.04it/s]


Saved: ../datasets//classificator/germline_v/train/train_germline_pairing_alpha-1000.csv
22 out of 112 of only zeros


...

Saved: ../datasets//classificator/germline_v/val/val_germline_pairing_alpha-1000.csv
20 out of 112 of only zeros


100%|██████████| 20/20 [00:00<00:00, 2844.37it/s]


Retrieve sequences


100%|██████████| 20/20 [00:00<00:00, 751.26it/s]


Saved: ../datasets//classificator/germline_v/test/test_germline_pairing_alpha-1000.csv


### Random pairing

In [50]:
import random_pairing

In [51]:
NUMBER = None

In [52]:
COMPUTE_NEW = True

In [ ]:
if COMPUTE_NEW:
    importlib.reload(random_pairing)
    for split in ['train', 'val', 'test']:
        random_pairing.random_pairing(
            '{}/{}/{}_seq.csv'.format(GERMLINE_V_DIR, split, split),
            '{}/{}/{}_germ'.format(GERMLINE_V_DIR, split, split),
            NUMBER, ALPHA
        )

In [10]:
if COMPUTE_NEW:
    importlib.reload(random_pairing)
    random_pairing.random_pairing('{}/classificator/train/train_seq.csv'.format(DATASET_DIRECTORY),
                                 '{}/classificator/train_random/train_random.csv'.format(DATASET_DIRECTORY), 
                                  NUMBER)

Sampling 716325 random pairs
Saved: ../datasets/new/classificator/train_random/train_random.csv


In [11]:
if COMPUTE_NEW:
    importlib.reload(random_pairing)
    random_pairing.random_pairing('{}/classificator/val/val_seq.csv'.format(DATASET_DIRECTORY),
                                 '{}/classificator/val_random/val_random.csv'.format(DATASET_DIRECTORY), 
                                  NUMBER)

Sampling 271233 random pairs
Saved: ../datasets/new/classificator/val_random/val_random.csv


In [12]:
if COMPUTE_NEW:
    importlib.reload(random_pairing)
    random_pairing.random_pairing('{}/classificator/test/test_seq.csv'.format(DATASET_DIRECTORY),
                                 '{}/classificator/test_random/test_random.csv'.format(DATASET_DIRECTORY), 
                                  NUMBER)

Sampling 369597 random pairs
Saved: ../datasets/new/classificator/test_random/test_random.csv


### Merge positive and negative

In [ ]:
import merge

In [ ]:
COMPUTE_NEW = True

In [ ]:
# Random dataset
if COMPUTE_NEW:
    importlib.reload(merge)
    merge.merge('{}/classificator/train/train_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/val/val_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/test/test_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test.csv'.format(DATASET_DIRECTORY))

In [ ]:
# Germline only V dataset
if COMPUTE_NEW:
    importlib.reload(merge)
    merge.merge('{}/classificator/train/train_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/val/val_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/test/test_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test.csv'.format(DATASET_DIRECTORY))

In [ ]:
# Germline dataset
if COMPUTE_NEW:
    importlib.reload(merge)
    merge.merge('{}/classificator/train/train_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/train_random/train.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/val/val_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/val_random/val.csv'.format(DATASET_DIRECTORY))
    merge.merge('{}/classificator/test/test_seq.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test_random.csv'.format(DATASET_DIRECTORY),
                '{}/classificator/test_random/test.csv'.format(DATASET_DIRECTORY))

### Test dataset

In [3]:
test_df = pd.read_csv(SEQUENCES_VAL, index_col=0)
light_sequences = test_df[['pair_id', 'light_id', 'light']].drop_duplicates().reset_index(drop=True)
light_sequences = light_sequences.sample(len(light_sequences))

In [4]:
import Levenshtein
from tqdm import tqdm
from itertools import combinations
from Bio import Align

from matplotlib import pyplot as plt

In [5]:
ls = light_sequences['light'].drop_duplicates().sample(10000).to_numpy()
sim = []
for x, y in combinations(ls, 2):
    sim.append(Levenshtein.ratio(x, y))

In [6]:
print(np.mean(sim), np.std(sim), np.min(sim), np.max(sim))

0.6231716007187188 0.11292343526922217 0.3677130044843049 0.9956331877729258


In [7]:
def create_pairs(light_sequences, threshold, sim_func):
    pairs = []
    index = 0
    for _, r in tqdm(light_sequences.iterrows(), total=len(light_sequences)):
        found = False
        #print('searching a fella for seq', r['light_id'])
        while not found:
            sim = sim_func(r['light'], light_sequences.iloc[index, 2])
            if sim < threshold:
                found = True
                pairs.append((r['pair_id'], r['light_id'], light_sequences.iloc[index, 1]))
            index += 1
            if index == len(light_sequences): 
                index = 0
        #pairs.append((r['light_id'], light_sequences.iloc[index, 0]))
    return pairs

def create_pairs_random(light_sequences):
    pairs = []
    sampled = light_sequences.sample(len(light_sequences))
    for (_, r1), (_, r2) in zip(light_sequences.iterrows(), sampled.iterrows()):
        print(r1, r2)
    
        
        
pairs_list = create_pairs(light_sequences, np.mean(sim) - np.std(sim), Levenshtein.ratio)

#create_pairs_random(light_sequences)

100%|██████████████████████████████████| 271230/271230 [06:35<00:00, 685.57it/s]


In [8]:
pairs = pd.DataFrame({
    'pair_id': [x for x, _, _ in pairs_list],
    'positive_light': [x for _, x, _ in pairs_list],
    'negative_light': [x for _, _, x in pairs_list]
})

In [9]:
df_pos = pd.merge(light_sequences[['light_id', 'light']].drop_duplicates(), 
                  pairs, 
                  left_on='light_id', right_on='positive_light', how='right').rename({
    'light_id': 'light_id_pos',
    'light': 'light_pos'
}, axis=1)[['pair_id', 'light_id_pos', 'light_pos']]

In [10]:
df_neg = pd.merge(light_sequences[['light_id', 'light']].drop_duplicates(), 
                  pairs, 
                  left_on='light_id', right_on='negative_light', how='right').rename({
    'light_id': 'light_id_neg',
    'light': 'light_neg'
}, axis=1)[['pair_id', 'light_id_neg', 'light_neg']]

In [11]:
df = pd.merge(df_pos, df_neg)
df = test_df[['pair_id', 'heavy_id', 'heavy']].merge(df)
df.to_csv('{}/val.csv'.format(TEST_DIRECTORY))